## 3. 转换器架构
这一章主要介绍转换器架构。虽然整体来说比较难理解，但它是生成式AI取得突破的根本原因，所以将其放在较靠前的章节。但从应用的角度来说，不理解转换器架构也没有太大关系。所以建议读者可以先粗读一遍，不管理解不理解都可以直接进入后续章节的学习。
学习转换器架构的关键是理解**多头自注意力（Multi-Head Self-Attention）**机制，其本质是以更为精准的向量来表示文本的语义，从而提升生成式AI的处理效率。

### 3.1 文本向量化
以向量的形式表示文本是AI能够理解人类语言的基础，但文本的向量化并不是简单地将文本中的每个字符映射到一个数值来组成向量，而是先将文本拆分为单词，再对单词进行向量化，最后将单词向量进行拼接，得到最终的文本向量。

#### 3.1.1 分词
AI中的单词或词语并不一定与自然语言中的单词或词语完全一致，它们本质上是出现频率较高的字符组合。所以为了与自然语言中的单词或词语相区分，在AI中一般将它们称为词元（Tokens）。绝大多数情况下，词元与单词或词语一致，但有些词元可能与单词或词语不同。比如生成式AI的英文“generative AI”中，generative就不是一个词元，而genera却是一个词元。因为在generative，generation，generator，general等等单词中都有genera这个字符组合，所以它的出现频率就更高一些，会被机器学习算法识别为词元。
这一小节主要介绍了三种常用的分词算法，即BPE（Byte-Pair Encoding）、WordPiece、SentencePiece。它们具体实现虽然不难但也不是特别容易理解，读者可以大致了解它们的基本思想即可。

#### 3.1.2 分词器
transformers库使用分词器（Tokenizer）负责分词，可通过AutoTokenizer加载预训练模型的分词器。本小节通过加载不同模型的分词器，向读者展示了不同分词算法处理词元的结果。
1. BERT模型采用了WordPiece算法
2. GPT-2模型采用了BPE算法
3. T5使用的分词算法为SentencePiece算法
使用下面的代码并替换模型名称，即可查看不同模型分词器的处理结果：

In [ ]:
from transformers import AutoTokenizer
# 选择预训练模型的名称
model_name = "bert-base-uncased"
# 加载预训练的分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 输出分词器的类型
print(tokenizer.tokenize("I love generative AI"))
# 获取BERT词汇表
voc = tokenizer.get_vocab()
# 查看genera和generate在词汇表中的数值ID
print(voc["genera"], voc["generate"])

#### 3.1.3  独热向量与嵌入向量
词元的向量形式有两种，一种是独热向量，另一种是嵌入向量。独热（One Hot）向量使用一个与词汇表长度相同的向量表示词元，整个向量中仅词元对应位置的分量为1，其它分量则全部为0。独热向量可以体现词元语义的独立性，但也存在着明显的问题。其一，独热向量是典型的稀疏向量，对于大模型数以万计的词汇表来说是巨大的空间浪费。另一个问题就是独热向量两两正交，无法体现词元之间在语义上的相似性。词嵌入（Word Embedding）是对独热向量的一种改进，它将词元映射到一个低维的语义空间中，每个维度可以看成是词元某一语义特征的强度，相似词元可以在相关维度上有相近似的强度，从而体现词元之间在某一语义特征上的相似性。所以词嵌入向量可以更准确地反映词元的语义相似性，是更适合AI处理的向量化文本输入。
本节举了一个非常有名的词元向量运算公式，可以让我们对词嵌入有一个直观的感知，那就是“king - man + woman ≈ queen”。理解了这个例子，对于文本向量化的理解就会更加清晰。

### 3.2 词嵌入算法简介*
本节介绍了两种词嵌入算法，即Word2Vec和GloVe。同时还介绍了一种降维算法t-SNE，用于可视化高维向量空间。

#### 3.2.1 嵌入矩阵
这一小节看似是在介绍嵌入矩阵，但其实是在介绍经典的获取词元嵌入向量的算法。
分词算法最终会生成一个包含所有词元的词汇表，词嵌入算法最终得到的是嵌入矩阵（Embedding Matrix）。嵌入矩阵使用一个整列（或整行）保存某一词元的嵌入向量，所以它的列数（或行数）就应该与词汇表的长度相同，而行数（或列数）则应该与嵌入向量的维度相同。模型将词元转换为嵌入向量时，只要使用嵌入矩阵与词元的独热向量相乘，得到的最终结果就是词元的嵌入向量。

早期生成嵌入矩阵的过程就是训练神经网络的过程，该神经网络以预测语句下一个词元为目标。所以它的输入是一条不完整语句的前序词元，而输出则是语句下一个词元在词汇表上的概率分布。嵌入矩阵虽然不是神经网络的一部分，但它会在训练过程中跟随参数一起调整。神经网络在训练过程会根据预测的误差确定调整方向，当误差达到可接受范围时结束训练，嵌入矩阵也在此时最终确立。

![嵌入矩阵训练过程](../res/images/classic_word_embedding.png)

#### 3.2.2 Word2Vec
Word2Vec是对嵌入矩阵训练过程的算法优化，它不以完整句子作为预测下一个词元的上下文，而是以单个词元作为预测下一个词元的上下文。Word2Vec首先确定一个中心词元做为输入，然后再在一个指定大小的窗口内选择另一个词元作为预测对象，包括skip-gram和cbow两种方法。下面的代码片段是使用python库gensim实现Word2Vec的例子：

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
# 示例文本数据
sentences = [
    ["I", "love", "machine", "learning"],
    ["I", "love", "deep", "learning"],
    ["I", "love", "NLP"],
    ["Word2Vec", "is", "a", "great", "tool"],
    ["Gensim", "is", "a", "useful", "library"],
]
# 训练Word2Vec模型
model = Word2Vec(sentences=sentences, vector_size=100, 
                 window=5, min_count=1, workers=4)
# 获取词汇表中的单词
words = list(model.wv.index_to_key)
print("Vocabulary:", words)
# 获取单词向量
vector = model.wv['machine']
print("Vector for 'machine':", vector)
# 找到与给定单词最相似的单词
similar_words = model.wv.most_similar('machine')
print("Words most similar to 'machine':", similar_words)

#### 3.2.3  GloVe
GloVe（Global Vectors for Word Representation）是对词嵌入矩阵训练过程的进一步优化，它基于一个基本的假设，那就是两个词元如果经常一起出现，它们的含义也就会或多或少有一些关联性。所以将最小化词元相似性与共现频率的误差做为目标，通过机器学习的方式就可通过多轮训练得到合适的嵌入向量。

#### 3.2.3  t-SNE
t-SNE是将每个向量与其它向量的相似性转化成了概率分布，并同时在高维空间和低维空间生成向量相似性的概率分布。最常用的实现了t-SNE的库是scikit-learn，下面的代码片段展示了使用t-SNE降维的过程：

In [ ]:
from nlp_util import to_embedding
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 定义需要降维的词汇
words=['king','queen','man','woman','apple','orange','fruit','computer','keyboard','mouse']
word_embeddings=to_embedding(words)

# 使用 t-SNE 将高维词嵌入降维到二维
tsne = TSNE(n_components=2, perplexity=2, random_state=0)
word_embeddings_2d = tsne.fit_transform(word_embeddings)

# 可视化结果
plt.figure(figsize=(10, 6))

for i, word in enumerate(words):
    plt.scatter(word_embeddings_2d[i, 0], word_embeddings_2d[i, 1])
plt.annotate(word, xy=(word_embeddings_2d[i, 0], word_embeddings_2d[i, 1]), xytext=(3*len(word), 2), textcoords='offset points', ha='right', va='bottom')

plt.show()

上述代码绘制的坐标图如下，从中可以清晰观察出数据点之间的距离关系，以及数据点之间的相似度关系：
![t-sne](../res/images/t-sne.png)

### 3.3  多头自注意力机制